<a href="https://colab.research.google.com/github/lukipuki0/GWO/blob/main/GWO_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.special import gamma
import pandas as pd
import matplotlib.pyplot as plt
import random
import math

# Librerias para CNN
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
import datetime
import time
import os


#link github: https://github.com/lukipuki0/GWO
# Configuración de visualización de Pandas  (No para HPC)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', 200)
#pd.set_option('display.max_colwidth', None)

#************************************************************************************************************
# *************************FUNCION PARA CNN *****************************************************************
def train_cnn_model(num_conv_layers, base_filter_value, use_batch_norm, lr, batch_size, epochs):
    start_time = time.time()  # inicio tiempo
    now = datetime.datetime.now()
    formatted_date = now.strftime('%Y%m%d_%H%M%S')
    #Directorio donde se guardarán las ejecuiones de CNN --cambiar nombre segun configuracion de esquema a aprobar

    base_directory = 'Esquema1'  # Ruta directa en HPC
    #base_directory='/content/drive/MyDrive/Colab Notebooks/Esquema1'

    folder_name = os.path.join(base_directory, f'ejecucion_cnn_{formatted_date}')

    # Revisar existencia de directorio
    os.makedirs(folder_name, exist_ok=True)
    log_file_path = os.path.join(folder_name, 'training_log.txt')

    class CustomCNN(nn.Module):
        def __init__(self, num_conv_layers, base_filter_value, use_batch_norm):
            super(CustomCNN, self).__init__()
            layers = []
            in_channels = 3  # Assuming input images have 3 channels (RGB)
            current_filters = base_filter_value

            for i in range(num_conv_layers):
                layers.append(nn.Conv2d(in_channels, current_filters, kernel_size=3, padding=1))
                if use_batch_norm:
                    layers.append(nn.BatchNorm2d(current_filters))
                layers.append(nn.ReLU(inplace=True))
                layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
                in_channels = current_filters
                current_filters *= 2  # Double the number of filters for the next layer

            self.features = nn.Sequential(*layers)
            self.fc1 = nn.Linear(in_channels * (64 // 2**num_conv_layers)**2, 1024)
            self.dropout = nn.Dropout(0.5)
            self.fc2 = nn.Linear(1024, 2)

        def forward(self, x):
            x = self.features(x)
            x = x.view(x.size(0), -1)
            x = F.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.fc2(x)
            return F.log_softmax(x, dim=1)

    # Data loading and transformation
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
    ])

    #Ruta de acceso local en colab
    #train_path = '/content/drive/MyDrive/Colab Notebooks/dataset/train'
    #valid_path ='/content/drive/MyDrive/Colab Notebooks/dataset/valid'
    #test_path = '/content/drive/MyDrive/Colab Notebooks/dataset/test'

    #Ruta de acceso en HPC
    train_path = '../../Datasets/dataset_experimental/train'
    valid_path = '../../Datasets/dataset_experimental/valid'
    test_path = '../../Datasets/dataset_experimental/test'

    train_data = datasets.ImageFolder(train_path, transform=transform)
    valid_data = datasets.ImageFolder(valid_path, transform=transform)
    test_data = datasets.ImageFolder(test_path, transform=transform)

    train_loader = DataLoader(train_data, batch_size, shuffle=True)
    valid_loader = DataLoader(valid_data, batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size, shuffle=False)

    model = CustomCNN(num_conv_layers, base_filter_value, use_batch_norm)
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)

    train_loss_history, valid_loss_history, valid_accuracy_history = [], [], []

    with open(log_file_path, 'w') as log_file:
        print('\nResultados por epochs')
        for epoch in range(epochs):
            model.train()
            total_train_loss = 0
            for data, target in train_loader:
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                total_train_loss += loss.item() * data.size(0)

            average_train_loss = total_train_loss / len(train_loader.dataset)
            train_loss_history.append(average_train_loss)

            model.eval()
            total_valid_loss, valid_correct, total_valid_samples = 0, 0, 0
            with torch.no_grad():
                for data, target in valid_loader:
                    output = model(data)
                    loss = criterion(output, target)
                    total_valid_loss += loss.item() * data.size(0)
                    pred = output.argmax(dim=1, keepdim=True)
                    valid_correct += pred.eq(target.view_as(pred)).sum().item()
                    total_valid_samples += data.size(0)

            average_valid_loss = total_valid_loss / total_valid_samples
            valid_accuracy = 100. * valid_correct / total_valid_samples
            valid_loss_history.append(average_valid_loss)
            valid_accuracy_history.append(valid_accuracy)
            log_msg = f'Epoch {epoch+1}/{epochs}, Training Loss: {average_train_loss:.4f}, Validation Loss: {average_valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.2f}%\n'
            log_file.write(log_msg)
            print(log_msg)

        # Calculate the final average values
        final_average_valid_loss = sum(valid_loss_history) / len(valid_loss_history)
        final_average_valid_accuracy = sum(valid_accuracy_history) / len(valid_accuracy_history)

        # Calculate total execution time
        end_time = time.time()
        total_execution_time = end_time - start_time

        # Convert execution time to hours, minutes, and seconds
        hours, rem = divmod(total_execution_time, 3600)
        minutes, seconds = divmod(rem, 60)
        time_formatted = f"{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

        # Log final results
        final_log_msg = f'\nFinal Average Validation Loss: {final_average_valid_loss:.4f}\n'
        final_log_msg += f'Final Average Validation Accuracy: {final_average_valid_accuracy:.2f}%\n'
        final_log_msg += f'Total Execution Time: {time_formatted} (hh:mm:ss)\n'
        log_file.write(final_log_msg)
        print(final_log_msg)

    # Plot training and validation loss
    plt.figure(figsize=(10, 5))
    plt.plot(train_loss_history, label='Training Loss')
    plt.plot(valid_loss_history, label='Validation Loss')
    plt.title('Training and Validation Loss per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(folder_name, 'loss_plot.png'))
    plt.close()

    # Plot validation accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(valid_accuracy_history, label='Validation Accuracy')
    plt.title('Validation Accuracy per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.savefig(os.path.join(folder_name, 'accuracy_plot.png'))
    plt.close()

    return final_average_valid_loss, final_average_valid_accuracy, total_execution_time




# Ejecucion del algoritmo GWO
start_time_cs = time.time()
best_nest, best_value, best_accuracy, best_loss, best_time,history, best_nest_per_iteration,best_accuracy_per_iteration, best_loss_per_iteration, best_execution_time_per_iteration, best_value_per_iteration = ejecutarGWO(num_lobos, dimension, maxIter, min_lim, max_lim, w1, w2, w3)
end_time_cs = time.time()



In [ ]:
import numpy as np
import random
import time
import os

#************************************************************************************************************
#**********************************  G R E Y   W O L F   O P T I M A Z E R  *********************************

# Grey Wolf Optimizer (GWO)
def gwo_optimize(wolves, max_iter, w1, w2, w3):

    #guardar el mejor valor durante la ejecucion
    history = []
    best_wolf = None
    best_value = float('inf')
    best_accuracy = 0
    best_loss = 0
    best_time = 0

    #Historial de datos por iteracion
    best_accuracy_per_iteration = []
    best_loss_per_iteration = []
    best_execution_time_per_iteration = []
    best_value_per_iteration = []
    best_wolf_per_iteration=[]

    #Ruta para HPC
    history_file_path = os.path.join('Esquema1/Historial', 'history.txt')


    with open(history_file_path, 'w') as f:
        f.write("")

    #calcular antes de los movimientos y actualizar lobos
    for wolf in wolves:
        final_average_valid_loss, final_average_valid_accuracy, total_execution_time = train_cnn_model(
            num_conv_layers=wolf['num_layers'],
            base_filter_value=wolf['num_filters'],
            use_batch_norm=wolf['batch_norm'],
            lr=wolf['lr'],
            batch_size=wolf['batch_size'],
            epochs=wolf['epochs']
        )
        current_value = w1 * (1 - final_average_valid_accuracy / 100 + w2 * final_average_valid_loss + w3 * total_execution_time)
        fitness_scores.append(current_value)

        print("\nLobo actual", wolf)
        print("\nAverage accuracy", final_average_valid_accuracy)
        print("\nAverage Loss", final_average_valid_loss)
        print("\nValor funcion objetivo", current_value)
        print("\nTiempo total en segundos ", total_execution_time)

        # Guardar historial
        if current_value < best_value:
            best_value = current_value
            best_accuracy = final_average_valid_accuracy
            best_loss = final_average_valid_loss
            best_time = total_execution_time
            best_wolf = wolf




    for t in range(max_iter):

        print("\n**********************************************  ITERACION ", i+1, " **********************************************")
        a = 2 - t * ((2) / maxIter)  # Disminución de a linealmente de 2 a 0

        fitness_scores = []

        # Ordenar lobos según fitness
        sorted_wolves = sorted(zip(fitness_scores, wolves), key=lambda x: x[0])
        alpha, beta, delta = sorted_wolves[0][1], sorted_wolves[1][1], sorted_wolves[2][1]

        # Actualizar posiciones de los lobos restantes
        for i, wolf in enumerate(wolves):
            for param in wolf:

                # Ignorar la actualización de Batch Norm (parámetro discreto)
                if param == 'batch_norm':
                    continue  # No actualizar el parámetro batch_norm (discreto)

                if isinstance(wolf[param], (int, float)):
                    r1, r2 = random.random(), random.random()
                    A1 = 2 * a * r1 - a
                    C1 = 2 * r2
                    D_alpha = abs(C1 * alpha[param] - wolf[param])
                    X1 = alpha[param] - A1 * D_alpha

                    r1, r2 = random.random(), random.random()
                    A2 = 2 * a * r1 - a
                    C2 = 2 * r2
                    D_beta = abs(C2 * beta[param] - wolf[param])
                    X2 = beta[param] - A2 * D_beta

                    r1, r2 = random.random(), random.random()
                    A3 = 2 * a * r1 - a
                    C3 = 2 * r2
                    D_delta = abs(C3 * delta[param] - wolf[param])
                    X3 = delta[param] - A3 * D_delta

                    # Actualización de la posición
                    wolf[param] = (X1 + X2 + X3) / 3


    return alpha  # El mejor conjunto de hiperparámetros

# Inicializar población de lobos (soluciones aleatorias)
def initialize_population(param_ranges):
    wolf = {}
    for param, options in param_ranges.items():
        if isinstance(options, list):  # Si las opciones son una lista
            wolf[param] = random.choice(options)  # Selecciona un valor aleatorio de la lista
        elif isinstance(options, tuple):  # Para rangos num�ricos, como 'lr' y epochs
            wolf[param] = round(random.uniform(options[0], options[1]), 4)  # Genera un valor flotante aleatorio para lr, redondeado en 4 decimales
            if param == "epochs":
                wolf[param] = int(wolf[param])  # Asegurar que epochs sea un entero
    return wolf



# Definición de los hiperparámetros con sus rangos
param_ranges = {
    "num_layers": [2, 3, 4, 5],  # Número de capas convolucionales
    "num_filters": [16, 32, 64, 128, 256],  # Número de filtros en cada capa
    "batch_norm": ["true", "false"],  # Usar o no Batch Normalization
    "epochs": (20, 50),  # Número de épocas
    "batch_size": [16, 32, 64],  # Tamaño del lote
    "lr": (0.0001, 0.01)  # Tasa de aprendizaje
}

# Parámetros de la ejecución
nwolves = 10      # Tamaño de la población (lobos)
maxIter = 10       # Número de iteraciones

w1 = 0.85
w2 = 0.1
w3 = 0.05

# Inicialización de la población de lobos
wolves = [initialize_population(param_ranges) for _ in range(nwolves)]

# Ejecucion del algoritmo GWO
start_time_cs = time.time()
best_wolf, best_value, best_accuracy, best_loss, best_time,history, best_wolf_per_iteration,best_accuracy_per_iteration, best_loss_per_iteration, best_execution_time_per_iteration, best_value_per_iteration = ejecutarGWO(wolves,maxIter, w1, w2, w3)
end_time_cs = time.time()

best_wolf = gwo_optimize(wolves,maxIter, w1, w2, w3)

# Guardar los hiperparámetros óptimos obtenidos
print("Mejor conjunto de hiperparámetros encontrados:", best_wolf)


In [3]:
import random
# Inicializar población de lobos (soluciones aleatorias)


# Inicializar población de lobos (soluciones aleatorias)


def initialize_population(param_ranges):
    wolf = {}
    for param, options in param_ranges.items():
        if isinstance(options, list):  # Si las opciones son una lista
            wolf[param] = random.choice(options)  # Selecciona un valor aleatorio de la lista
        elif isinstance(options, tuple):  # Para rangos num�ricos, como 'lr' y epochs
            wolf[param] = round(random.uniform(options[0], options[1]), 4)  # Genera un valor flotante aleatorio para lr, redondeado en 4 decimales
            if param == "epochs":
                wolf[param] = int(wolf[param])  # Asegurar que epochs sea un entero
    return wolf




# Definición de los hiperparámetros con sus rangos
param_ranges = {
    "num_layers": [2, 3, 4, 5],  # Número de capas convolucionales
    "num_filters": [16, 32, 64, 128, 256],  # Número de filtros en cada capa
    "batch_norm": ["true", "false"],  # Usar o no Batch Normalization
    "epochs": (20, 50),  # Número de épocas
    "batch_size": [16, 32, 64],  # Tamaño del lote
    "lr": (0.0001, 0.01)  # Tasa de aprendizaje
}
nwolves = 10
# Inicialización de la población de lobos
wolves = [initialize_population(param_ranges) for _ in range(nwolves)]
#wolves = initialize_population(nwolves, param_ranges)

wolves


[{'num_layers': 5,
  'num_filters': 16,
  'batch_norm': 'false',
  'epochs': 49,
  'batch_size': 32,
  'lr': 0.0081},
 {'num_layers': 4,
  'num_filters': 256,
  'batch_norm': 'false',
  'epochs': 22,
  'batch_size': 64,
  'lr': 0.0092},
 {'num_layers': 3,
  'num_filters': 128,
  'batch_norm': 'true',
  'epochs': 35,
  'batch_size': 64,
  'lr': 0.0001},
 {'num_layers': 2,
  'num_filters': 16,
  'batch_norm': 'true',
  'epochs': 23,
  'batch_size': 32,
  'lr': 0.0033},
 {'num_layers': 5,
  'num_filters': 64,
  'batch_norm': 'false',
  'epochs': 26,
  'batch_size': 16,
  'lr': 0.0075},
 {'num_layers': 5,
  'num_filters': 256,
  'batch_norm': 'false',
  'epochs': 39,
  'batch_size': 64,
  'lr': 0.0017},
 {'num_layers': 5,
  'num_filters': 64,
  'batch_norm': 'true',
  'epochs': 22,
  'batch_size': 16,
  'lr': 0.0004},
 {'num_layers': 3,
  'num_filters': 128,
  'batch_norm': 'false',
  'epochs': 23,
  'batch_size': 64,
  'lr': 0.0062},
 {'num_layers': 5,
  'num_filters': 256,
  'batch_norm'